Tendencia anual de consumo

In [1]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import regexp_replace, col, sum, year, month

spark = SparkSession.builder \
    .appName("RankingProdutos") \
    .getOrCreate()

Using Spark's default log4j profile: org/apache/spark/log4j2-defaults.properties
25/06/17 18:49:26 WARN Utils: Your hostname, DESKTOP-EDEM2DH, resolves to a loopback address: 127.0.1.1; using 10.255.255.254 instead (on interface lo)
25/06/17 18:49:26 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Using Spark's default log4j profile: org/apache/spark/log4j2-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/06/17 18:49:26 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [2]:
df_atual = spark.read.parquet("../../datalake/silver/Liquidos_Vendas_Atual")
df_historico = spark.read.parquet("../../datalake/silver/Liquidos_Vendas_Historico_2007_a_2017")


In [3]:
df_atual_sel = df_atual.select(
    col("ano").cast("int").alias("ano"),
    regexp_replace("quantidade_de_produto_(mil_m3)", ",", ".")
        .cast("double").alias("quantidade")
)

df_historico_sel = df_historico.select(
    col("ano").cast("int").alias("ano"),
    regexp_replace("volume_vendido", ",", ".")
        .cast("double").alias("quantidade")
)

df_vendas = df_atual_sel.unionByName(df_historico_sel)

df_anual = df_vendas.groupBy("ano").agg(sum("quantidade").alias("volume_total"))

In [4]:
df_anual.write.mode("overwrite").parquet("../../datalake/gold/tendenia_anual_consumo")